In [ ]:
import torch, librosa
from muq import MuQ

device = 'cuda'
wav, sr = librosa.load("/data2/zhouyz/rec/MSD/MSD_2/save/AA/TRAAJEN144D1BB07C0.mp3", sr = 24000)
wavs = torch.tensor(wav).unsqueeze(0).to(device) 

# This will automatically fetch the checkpoint from huggingface
muq = MuQ.from_pretrained("OpenMuQ/MuQ-large-msd-iter")
muq = muq.to(device).eval()

with torch.no_grad():
    output = muq(wavs, output_hidden_states=True)

print('Total number of layers: ', len(output.hidden_states))
print('Feature shape: ', output.hidden_states[0].shape) # L: 13

/user/zhouyz/anaconda3/envs/zhouyz/lib/python3.9/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


Total number of layers:  13
Feature shape:  torch.Size([1, 3993, 1024])


In [9]:
import numpy as np
all_features = []
for l in range(1, len(output.hidden_states)):
    feature_i = output.hidden_states[0].detach().cpu().numpy().squeeze().mean(axis=0)
    all_features.append(feature_i)

all_features = np.array(all_features)
print('All features shape: ', all_features.shape) # (13, 1024)

All features shape:  (12, 1024)
